# Large Transactions in the Mempool
Author: Evan Azevedo  
Organization: Amberdata  

## Table of Contents
* Finding Whales on Ethereum from the past year
* Exploring their mempools
* Interpreting the results

In [4]:
# load the packages
import os
import asyncio
import requests
import logging
import warnings
import datetime as dt
import json
import ssl

import websocket
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dotenv import load_dotenv

In [14]:
# helper functions
def get_key():
    "Get the API key from an .env file"
    if ".env" not in os.listdir("../"):
        print("Configuring API Key...")
        key = input("Amberdata API Key: ")
        with open("../.env", "w") as f:
            f.write(f"AMBERDATA_API_KEY={key}\n")
    load_dotenv(verbose=True)
    return {
        "AMBERDATA_API_KEY": os.getenv("AMBERDATA_API_KEY")
    }

def get_response(url, headers=None, queryString=None):
    "Get the REST response from the specified URL"
    if not headers:
        headers = {'x-api-key': api_key["AMBERDATA_API_KEY"]}
    if queryString:
        response = requests.request("GET", url, headers=headers, params=queryString)
    else:
        response = requests.request("GET", url, headers=headers)
    response = json.loads(response.text)
    try:
        if response["title"] == "OK":
            return response["payload"]
    except Exception:
        return None

    
api_key = get_key()["AMBERDATA_API_KEY"]

## Whale Hunting

In [ ]:
AMBERDATA_API_KEY = os.environ['AMBERDATA_API_KEY']
AMBERDATA_BASE = 'https://web3api.io/api/v1'
AMBERDATA_BASE_MARKET = AMBERDATA_BASE + '/market'
AMBERDATA_WEBSOCKET_BASE = 'wss://ws.web3api.io'

In [13]:
api_key

{'AMBERDATA_API_KEY': 'UAK7c1702e541c3acb875a904ba47676f93'}

In [18]:
def on_open(ws):
    """Sends a message upon opening the websocket connection"""
    print('websocket {} was connected'.format(ws.url))
    ws.send(json.dumps({
        'jsonrpc': '2.0',
        'id': 1,
        'method': 'subscribe',
        'params': ['pending_transaction']
    }))
    
def on_message(ws, message):
    "reacts to messages from the websocket"
    json_message = json.loads(message)
    if json_message.get('params') and json_message.get('params').get('result'):
        result = json_message.get('params').get('result')
        print(result)
    

In [21]:
print(api_key)

UAK7c1702e541c3acb875a904ba47676f93


In [20]:
ws = websocket.WebSocketApp('wss://ws.web3api.io')
ws.header = {"x-api-key": api_key, "x-amberdata-blockchain-id":"bitcoin-mainnet"}
ws.on_open = on_open
ws.on_message = on_message
ws.run_forever(sslopt={'cert_reqs': ssl.CERT_NONE})
print('main end')

websocket wss://ws.web3api.io was connected
{'blockchainId': '408fa195a34b533de9ad9889f076045e', 'from': ['1DsHrAcA7rbNpeme7Pgko2kpcb39LHt4NB', '1EL1yJWkGhJkwi98XWHsanmk9ytmTFdSMY', '1xygXmyPbSrVgZdrUeaiH1WjV2c3v6qr2', '13e7JcqdM3RFWt5bnCLxjwNfJYzXLvdtnV', '1KWvJYwp2QDNT3jH73NyQ5LBQqXaEA38kb', '1AdPUWMhfHTBRSd6CtxC21DFdqg87K589f', '1MbGcabLHk5rhFk1RtaSHsVBnyxUBMA2sZ', '13Dn5gqrq5jRAuPvcpJhiWbURVUHrF7D5t', '1HWKdQwxrCrh8fRbLRJdN5jt5mb7WdUKpP', '1QH4ffD8vfk2qrkoJxzXhYFDoo3ezLxrq9', '13NP7YbbybyEz9cdPceueFWMP7jXrjjqiL', '1NKcT9svHcWPQthWdLu4Haozde7NSufQ7P', '1JemEFNjYz2o43PEF7os7xq2GnSUsuzCyC', '16pAm7Et8U78JMyKn52QAnrR5YYeDESvs6', '1MbGcabLHk5rhFk1RtaSHsVBnyxUBMA2sZ', '1BbW7TdVMMqGHZXyqad13kp98rJSJx2vrK'], 'hash': 'b6e7d9b71884eed8da83e74d2bada12867c9b0fe5601035e79e82553aeb3def2', 'inputValue': 5500000, 'inputs': [{'sequence': 4294967295, 'required_signatures': 1, 'type': 'pubkeyhash', 'addresses': ['1DsHrAcA7rbNpeme7Pgko2kpcb39LHt4NB'], 'value': 100000, 'index': 0, 'requiredSignatures':

{'blockchainId': '408fa195a34b533de9ad9889f076045e', 'from': ['1KmheFDStisf1ykAmD8hRgJ3pSFKpVLD9a', '18UXWQgbhave8TghwXwwP7e3kRsfYmBXvv', '1EX5EuCt1eeiZUCkCFruHAKgExya1c81fA', '1Er8G7TZzaYPQZ3C2Vh1RkQv6zSzaRiDMz', '13CFZtwfPmu72wxr4BK3W6MpiK5RnXu3vg', '153uJt3hWVEynwHecapTaU9rMRerGTtAoP', '1BHijDZkFX1kQi8VbcscZUPhpfYda1FuTM', '1DBwEb4gMmD8HMs71enexarHWWHUqsPnVd', '1CaUNq4CwYko1GXv87oWG84qfKDfCtzvVn', '16T3J3ZfN6jwotX1tTi8McYZPrrFVuNQxz', '1BfPYVPdPsZMMiKMrSbjnuCW4vxdRJyHj7', '14vyBJXpVMXzPJ7Ym2mGuswPHKkEPwBihi', '1T8BTuT6MU72Q6FVA5LLPfxJ4yCN5Xx4D', '1B63nS2Qnpp5UqysumSfkg8nPb7s9W2KZs', '1F9MqAraXmccx1igAFoqYwweJmUZpj5es2', '1Czn6U4yvYMJXTkg3vfejvobtjpAf3sZja'], 'hash': '4f31a4ad651e7e6fa849f0e54c4c879a1ef7053b205086b63a108bd2b055fcae', 'inputValue': 22486710, 'inputs': [{'sequence': 4294967295, 'required_signatures': 1, 'type': 'pubkeyhash', 'addresses': ['1KmheFDStisf1ykAmD8hRgJ3pSFKpVLD9a'], 'value': 800000, 'index': 0, 'requiredSignatures': 1, 'scriptAsm': '3044022037fd6a25094d99a6e

{'blockchainId': '408fa195a34b533de9ad9889f076045e', 'from': ['17oWjpMiEYxb3B4EhgVKX3PSQ8B1dV2dkc', '1JeqqkeN66XtA6s8diQWLeHWRNhA7KXe4s', '1GQbiafD5CCBDMLkSLbpyRJChrosoLrd7o', '18CzTdsgYdKbvYd4wPBWnvXoknde3jf59T', '19jdEDqzrdx7rCXpVuv1VbmhjNQbPvfZ5g', '174iEYweMKCUc398QAiKMHqyYAFzH3Na3K', '14okxVcAEuZ1qA6LpkceVR92kmQeufxB1M', '17FfQUXu8wHD22NSvWbJrNmrJUDnqWJZwF', '12BBwhR1GBWLyzf8uWTUq1kCjWaFhSGjUx', '14EzZDeztEhVNeHxeouDpdfVu99oPLsgv3', '17c4a6zHT5r6SpV3dW612ffFtdtFw8y5NW', '1CDiGo9Xm5wCbAXUeBAr5r9NsNVKvcBWtz', '1Dw6AKAXzdkDH9KLKMyBkkdVXYfVCJ9Fas', '14XQuE6sny5Dzgd1SB1ob7FkkU6ytDqQej', '17dtbr4J4iuTrczPtn9XZq9C9ux1rDDXYq', '1HEWVTPk129tVn7WVH2RGwe5xYatBZeP27'], 'hash': 'ccd135ad82155c53b510b544434eb67247c1de89103040cf722626826f031388', 'inputValue': 36496137, 'inputs': [{'sequence': 4294967295, 'required_signatures': 1, 'type': 'pubkeyhash', 'addresses': ['17oWjpMiEYxb3B4EhgVKX3PSQ8B1dV2dkc'], 'value': 4930601, 'index': 0, 'requiredSignatures': 1, 'scriptAsm': '30440220209a3e98368ae89

{'blockchainId': '408fa195a34b533de9ad9889f076045e', 'from': ['1ENG6yUtDqb5dmjkLni8vcFRAk1cvpGshB', '1PxfJZgeH7uz7MHQjPoYonnFifbnx16Tqh', '1JPoC2oU5Ruqc2wvgxkeYT99Rrswci5tfN', '1Nc8rwWAQeKkzvrusbo43EFZ9srurtaMf4', '1Hj1uPQt7z8zfwqFNcJSAnkZkurVRkKRmB', '1Dxx8zpeUBsH6haMQ4nzhjCwTWo2K7E2pw', '13nLffoAJ543j8Axe7aztnzVDcmp2upYKN', '1FuxWYHQjStEAkwxgA2Fu8CfehhuWsxvcV', '19w2xVyzytRF9QgeBpb7tfCPkdkrwitSfZ', '149yKT3NpKSMtWjKJtPec6mMA56P29efiK', '1DpUAQ8zuJ8ZbBwuj8bZtNEChUb3KbXjPD', '19yL5rAncWjH71eAJECcKZJhiY3B8s86Yx', '1MVPwqxcCwbW6o217QheokiSnGbva5PfJW', '1CfUU89bA1ZXhkq9DDRe3fw1q4c73LHEuA', '1G2N2AVeQhKkQC4QZRhyW2RacmMdKaa7GM', '14iCSWLbXN7KPE2itCdpdsYbdKUPiEbK8D'], 'hash': 'a4f281ccd3afcc8e1f08bceeaa8a1ea73e5f4ec886dac70635a9f571fa973d73', 'inputValue': 2463442, 'inputs': [{'sequence': 4294967295, 'required_signatures': 1, 'type': 'pubkeyhash', 'addresses': ['1ENG6yUtDqb5dmjkLni8vcFRAk1cvpGshB'], 'value': 45560, 'index': 0, 'requiredSignatures': 1, 'scriptAsm': '3044022037663e36564160bc26

{'blockchainId': '408fa195a34b533de9ad9889f076045e', 'from': ['1JGZDy9HzQuZqy8gWHLxkPYVLQTRYt7wdD', '15r9pVEKAMNDDEmsdxRsLp25yNP5wQ2E1X', '1ALH3sqNuDgEP4gbEHEqLN2ac7t3L6ohbL', '1E9KexFRC9ZimfYY52jU2aypxaCJZsGvfR', '1DFkpV47vSs5DDVXqV3Z9ux5awiV8rHLjT', '18QBPisMdFutSCkhCgST2sUEfAhBeCmRYq', '15EnVduYAgpSifufiDdU54MkmzJwZDPk9s', '1HdqsMDbh41XkXrmwrDyrHQ2EXDfjGJ2ad', '14MHzWHDKgVAwi4pVJzcNYEURX6fMN3QsQ', '1PL4Z17kS1B9dMVsMWptg8pigXrmBosLEf', '18kRtrwsB1euzYiwDfcs1TdiaZYSuRHmqf', '1Mty316MbzYHbtGmY3DiYohvFo5LvMvj4R', '1CPCN7YJ6N6pa2cTMwR7ibrhn3rqk9ffLo', '14x3sizQJHttBZgjmXXrQM3ftbGm6i1BVZ', '114YLtfiXJxfPEZP7FuDXoevPNY2rTpMbZ', '1JZYeN9ZLYJCqcMd1wMHdoo1vWvobessW'], 'hash': '66c8ad1949e7dcea94ff2ad3ef04b9f8f0a951da76cfc7872e67242b9c478550', 'inputValue': 77032641, 'inputs': [{'sequence': 4294967295, 'required_signatures': 1, 'type': 'pubkeyhash', 'addresses': ['1JGZDy9HzQuZqy8gWHLxkPYVLQTRYt7wdD'], 'value': 13216342, 'index': 0, 'requiredSignatures': 1, 'scriptAsm': '3044022023e6eaeb7ece4c9

...

{'blockchainId': '408fa195a34b533de9ad9889f076045e', 'from': ['1JQQcCTyy2eM3HQE7TF72kRpVAa45zsbya', '13HFmNvwAppXyDmJHnJMN6aE3zR7khNF9a', '16aczUPGbzq6BMWiQ8ULGaekRg9mVH3Ban', '15gFfZ1FVHJuP9T52FnrfNTgwHAq5WTj6n', '1E1ETKe6KH3iGY6Jzs5zXXuWEYh6ENB4oT', '1MDxchAsN7EWHhwkqpbF7SvK93VgGzbMS9', '16QyXq8XJSkASs8RmosR1e2Mv5BCybiwYn', '18zNimSyHeXejs5gZHVtqvHyNWmrSPirco', '1B1FwvCKhtT1EzZBNt1NZWSMKYKCGKZeRW', '1NedajJYzhCo4t4LmYZnnXR3omFEBW1Yk9', '1M5nmZCynTpwkW19U7mWPEe97wFMcDGzQc', '1CFdWRC4r3HR76idna49SDTnyTJZRWr1MK', '147UaUxNzxe6SQfCcNJ5dauep3fVqxCPzD', '1N5Sa1j4tNNCQtn8oYNoX1AZfNTP9secpo', '1ENfVkCDvbze8DSsG6pBL4pugvuR42wwdh', '1DWrEhM9RuUtMTVWG5TEwq1FwjZdYzEpNP'], 'hash': 'ad90e3f4fd41a44e3f72f7a9dd99a46dff4662eedb52289f4ff31f577b92c6d9', 'inputValue': 8680416, 'inputs': [{'sequence': 4294967295, 'required_signatures': 1, 'type': 'pubkeyhash', 'addresses': ['1JQQcCTyy2eM3HQE7TF72kRpVAa45zsbya'], 'value': 340747, 'index': 0, 'requiredSignatures': 1, 'scriptAsm': '3045022100fe8034c97bbb1e1

{'blockchainId': '408fa195a34b533de9ad9889f076045e', 'from': ['1EoXcNw7ZmM754JPj6R1N45tnHM6RkKRh1', '1PWZ2uk5usNfz83PaqpNmruw82KxhLhGsd', '1CS5nU3cPRHKQgvC22m2RadafVoNtNrtxH', '13fK6UPpLMNDAYd3Zipa6AcaxXYYBwzyc6', '1AF3du9KdExfYW6S7MXd6UfTp8oEEw996N', '1AfRuMBFz5FqpaHSZawKTeZoZBaxjVDmRJ', '1DaYVqkS3TzbGtL7wU1hXTrnw7VhtX2uPF', '1DuNfJmiRsswvGKCPn6ugBwYeSAVrEnnwv', '1AARMT5N5w61o4khCPsK7qW3jGKPyUj5En', '1322wjLqK6n62grnhpemyHS9MHHcsU6Uyj', '1C9DpRLp17S5oqi5ipPyUhpPQb4R6tSGhw', '1ARkQk79WLS2aD7jxVeDv9xaW9FYzYhmym', '1MyfSkEMtQt9WcZRN5kFN3ohk27SPtGbz7', '195Y9bS3cyW5124SUtLLyZncJYH9EBe1Xi', '1CZ58SWZ4xExXogJc7fGNV1TL6XZGCPNvN', '1CZ58SWZ4xExXogJc7fGNV1TL6XZGCPNvN'], 'hash': '3f41dbdb0ea0109a375d46749bbc6ed5ed0ccd7071934e16e2f55ceec2e0bae3', 'inputValue': 49858888, 'inputs': [{'sequence': 4294967295, 'required_signatures': 1, 'type': 'pubkeyhash', 'addresses': ['1EoXcNw7ZmM754JPj6R1N45tnHM6RkKRh1'], 'value': 10357568, 'index': 0, 'requiredSignatures': 1, 'scriptAsm': '3045022100a1788c4b6d20

{'blockchainId': '408fa195a34b533de9ad9889f076045e', 'from': ['17jpHTHUDBR5Y5DQeUC4CdtUQDdCtsfgWD', '14hecYnYF2i92brYSotsVjKaNdzMtCAfiK', '12wWjNHLZDdwgXP59KYi8aK852e32zxVnC', '1GvHqpA9n3n9icwaMWzD4eNVSPkKXNAwKP', '13HFmNvwAppXyDmJHnJMN6aE3zR7khNF9a', '1LJq4dXb4RXBpNh9ms42M8KouSt6bkHaRh', '16HtSx1bJG2D8iEza3Vxd67v4qST87jShT', '192MMtB7PLUHvM71C8yBzDjzRtPTc5mxF5', '1LXNunxdXh9kLg3Y4LvbT1SMd3Dfiu8UXD', '1LcrHx1Ti669Rs7HPa44Xfj5Db1GzjaxK3', '1LtWY7NTK4CHzaTqiEvbURKvnhmc8Vqo5P', '1B5BHxYHXSdBSumGfmXo2V27bUjgmec7r3', '1CPbtx3uGKyFvFdUE3ZD7qTuCjsRRq2859', '1KtAShUx6A8VhdaJzbFvvuGBBpueHzmEep', '16kEKW7HG8BXz8vvdtDYYCVrpoTEHS8AAL', '1KguxdXtQm5LmisUR8mC4wzPxp4rmTQJrD'], 'hash': '98376bdd644069b916b999c92883fc8eb3ffe7f2034ed2fd41a8e1435ba99bbc', 'inputValue': 4653051, 'inputs': [{'sequence': 4294967295, 'required_signatures': 1, 'type': 'pubkeyhash', 'addresses': ['17jpHTHUDBR5Y5DQeUC4CdtUQDdCtsfgWD'], 'value': 137300, 'index': 0, 'requiredSignatures': 1, 'scriptAsm': '3045022100de9fe5de457f2f5

{'blockchainId': '408fa195a34b533de9ad9889f076045e', 'from': ['156oVwth7M39crSQeM9eK9fTtoNTT1kWWZ', '14NMzTQgDue9GFUtcHmUoA9HrQc2MmKYjx', '14NMzTQgDue9GFUtcHmUoA9HrQc2MmKYjx', '18bdunrfPhP2xqwQ4kDDvtNS8YewKqAfRh', '14NMzTQgDue9GFUtcHmUoA9HrQc2MmKYjx', '1EAaVF3hYziCR3xZQEySwnPz6yJC78ndKv', '1LJq4dXb4RXBpNh9ms42M8KouSt6bkHaRh', '1Dgu3s8xejJYMgTdiJPj5U2JKybn23VuTF', '19m69gj9qrfswghUszZbzFtrorUfpZXeM4', '1BSjqv7EjYEAVdohbzDW89S4xgHDUparUy', '1EzhGK6ymP8PfsPkbqYwzx8K2XEWaj1uoz', '14KbFNSq5L2n7EDicafibkTpsd2XUD2Mor', '1ARU7JmwXiE6QDdZAFCuJY74AZDv93WoQB', '1FHo6t7WejHSbpzazjtkTt61n6UZm7YQCP', '1FcXUNcrcEGYqrcThBy64jdJ3PZQ2e5J91', '1L9RHoLPfUHfS6bRuGApoZ1eAgn5zCvvUg'], 'hash': 'be0d9cf611a1872e7dbb3f1f7bab34c4b5a90b7f410746b1b9bf9bd2c654bbb3', 'inputValue': 486539864, 'inputs': [{'sequence': 4294967295, 'required_signatures': 1, 'type': 'pubkeyhash', 'addresses': ['156oVwth7M39crSQeM9eK9fTtoNTT1kWWZ'], 'value': 300000000, 'index': 0, 'requiredSignatures': 1, 'scriptAsm': '3045022100cd2836b5b4

main end


In [13]:
url = "https://web3api.io/api/v2/transactions/"

querystring = {"status":"pending","size":"100","includeFunctions":"false",
               "includeLogs":"true","includeTokenTransfers":"false"}

headers = {
    'x-amberdata-blockchain-id': "ethereum-mainnet",
    'x-api-key': api_key["AMBERDATA_API_KEY"]
    }

response = get_response(url, headers, querystring)

df = pd.DataFrame(response["records"])

In [15]:
df.columns

Index(['blockNumber', 'confirmedAt', 'confirmedHash', 'createdAt', 'from',
       'gasLimit', 'gasPrice', 'hash', 'index', 'input', 'nonce',
       'occurrences', 'publicKey', 'r', 'raw', 's', 'state', 'to', 'updatedAt',
       'v', 'value', 'statusResult', 'timestamp'],
      dtype='object')